# 시금치 & 오이 가격 예측 모델 선정
*XGBoost, LightGBM, RandomForest 모델을 튜닝하고, 이들을 결합한 앙상블 모델(VotingRegressor)과 성능을 비교하여 최종 최적 모델을 선정합니다.*

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
import warnings

warnings.filterwarnings('ignore')

## 1. 데이터 불러오기 및 전처리

In [4]:
# 데이터 불러오기
cold_df = pd.read_excel('../data/mon_cold.xlsx')
wind_df = pd.read_excel('../data/mon_wind.xlsx')
hot_df = pd.read_excel('../data/mon_hot.xlsx')
price_df = pd.read_excel('../data/region_price.xlsx')
weather_df = pd.read_csv('../data/region_weather.csv')

# 데이터 변환 및 병합
cold_df_melted = cold_df.melt(id_vars=['지역'], var_name='날짜', value_name='한파발생')
wind_df_melted = wind_df.melt(id_vars=['지역'], var_name='날짜', value_name='태풍발생')
hot_df_melted = hot_df.melt(id_vars=['지역'], var_name='날짜', value_name='폭염발생')
price_df['날짜'] = pd.to_datetime(price_df['날짜'], errors='coerce').dt.strftime('%Y-%m')
weather_df['날짜'] = pd.to_datetime(weather_df['날짜'], errors='coerce').dt.strftime('%Y-%m')
merged_df = pd.merge(price_df, weather_df, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, cold_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, wind_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, hot_df_melted, on=['지역', '날짜'], how='left')
merged_df.dropna(subset=['평균가격'], inplace=True)
merged_df = merged_df[merged_df['평균가격'] > 0].copy()
merged_df.fillna(0, inplace=True)

# 특성 공학
merged_df['날짜'] = pd.to_datetime(merged_df['날짜'])
merged_df['연도'] = merged_df['날짜'].dt.year
merged_df['월'] = merged_df['날짜'].dt.month

## 2. 모델 학습, 튜닝 및 최종 비교

In [3]:
def find_best_model(df, vegetable_name):
    # 데이터 분리
    target_df = df[df['품목'] == vegetable_name].copy()
    target_df['지역'] = target_df['지역'].astype('category').cat.codes

    features = ['평균기온(°C)', '월합강수량(00~24h만)(mm)', '평균풍속(m/s)', '최심적설(cm)', '한파발생', '태풍발생', '폭염발생', '지역', '연도', '월']
    target = '평균가격'

    X = target_df[features]
    y = target_df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 및 하이퍼파라미터 그리드 정의
    rf = RandomForestRegressor(random_state=42)
    xgb = XGBRegressor(random_state=42)
    # LightGBM 경고를 숨기기 위해 verbose=-1 추가
    lgbm = LGBMRegressor(random_state=42, verbose=-1)

    param_grid_rf = {'n_estimators': [100, 200], 'max_depth': [5, 10]}
    param_grid_xgb = {'n_estimators': [100, 200], 'max_depth': [5, 7], 'learning_rate': [0.05, 0.1]}
    param_grid_lgbm = {'n_estimators': [100, 200], 'max_depth': [5, 7], 'learning_rate': [0.05, 0.1]}

    models_and_params = [
        ('RandomForest', rf, param_grid_rf),
        ('XGBoost', xgb, param_grid_xgb),
        ('LightGBM', lgbm, param_grid_lgbm)
    ]

    best_estimators = []
    results = {}

    print(f'--- {vegetable_name} 모델 튜닝 및 평가 시작 ---')

    # 각 개별 모델 튜닝 및 평가
    for name, model, params in models_and_params:
        print(f'{name} 모델 튜닝 중...')
        grid_search = GridSearchCV(estimator=model, param_grid=params, cv=3, n_jobs=-1, scoring='r2')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        results[name] = r2
        best_estimators.append((name, best_model))
        print(f'{name} 최적 파라미터: {grid_search.best_params_}')
        print(f'{name} R-squared: {r2:.4f}')

    # 앙상블 모델 (VotingRegressor) 생성 및 평가
    print('앙상블(Voting) 모델 평가 중...')
    voting_model = VotingRegressor(estimators=best_estimators)
    voting_model.fit(X_train, y_train)
    y_pred_voting = voting_model.predict(X_test)
    r2_voting = r2_score(y_test, y_pred_voting)
    results['VotingEnsemble'] = r2_voting
    print(f'Voting Ensemble R-squared: {r2_voting:.4f}')

    # 최종 결과 출력
    best_overall_model = max(results, key=results.get)
    print(f'--- {vegetable_name} 최종 결과 ---')
    for name, score in results.items():
        print(f'{name}: {score:.4f}')
    print(f'=> {vegetable_name}의 최종 최적 모델은 {best_overall_model} (R-squared: {results[best_overall_model]:.4f}) 입니다.')
    print('-'*50)

# 시금치와 오이에 대해 함수 실행
find_best_model(merged_df, '시금치')
find_best_model(merged_df, '오이')

--- 시금치 모델 튜닝 및 평가 시작 ---
RandomForest 모델 튜닝 중...
RandomForest 최적 파라미터: {'max_depth': 10, 'n_estimators': 100}
RandomForest R-squared: 0.6768
XGBoost 모델 튜닝 중...
XGBoost 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
XGBoost R-squared: 0.7044
LightGBM 모델 튜닝 중...
LightGBM 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
LightGBM R-squared: 0.6808
앙상블(Voting) 모델 평가 중...
Voting Ensemble R-squared: 0.7039
--- 시금치 최종 결과 ---
RandomForest: 0.6768
XGBoost: 0.7044
LightGBM: 0.6808
VotingEnsemble: 0.7039
=> 시금치의 최종 최적 모델은 XGBoost (R-squared: 0.7044) 입니다.
--------------------------------------------------
--- 오이 모델 튜닝 및 평가 시작 ---
RandomForest 모델 튜닝 중...
RandomForest 최적 파라미터: {'max_depth': 10, 'n_estimators': 100}
RandomForest R-squared: 0.7926
XGBoost 모델 튜닝 중...
XGBoost 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
XGBoost R-squared: 0.8513
LightGBM 모델 튜닝 중...
LightGBM 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 2